In [1]:
import os
import glob
import pickle
from datetime import datetime
import time
import dotenv
import pandas as pd

import requests
import requests.auth

import praw

from typing import List

import openai

import langchain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

from pydantic import BaseModel, Field, validator

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# load secrets from .env into environment variables
dotenv.load_dotenv()

langchain.__version__

'0.0.154'

- interrogate praw docs and write code to download from reddit
- do some language stuff and extract the songs
- interrogate spotipy write code to look up the uri
- interrogate spotipy, write code to get the playlist id
- interrogate spotipy, write code to update the playlist



In [ ]:
# models_obj = openai.Model.list()
# sorted([m['id'] for m in models_obj['data']])


In [3]:
# this doesn't work for some reason, end up in version hell
# loader = DirectoryLoader(hftcdir, glob="*.txt")
# d=loader.load_and_split()


ImportError: cannot import name 'open_filename' from 'pdfminer.utils' (/Users/drucev/opt/anaconda3/envs/llama/lib/python3.9/site-packages/pdfminer/utils.py)

In [2]:
hftcdir = '/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc'

documents = None
for f in glob.glob('%s/*' % hftcdir):

    print(f)
    if documents is None:
        documents = TextLoader(f).load()
    else:
        documents.extend(TextLoader(f).load())
    

/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20211511-all-BenjaminSaberin@distro.hftconnect.com-LZ2JJVVWUW.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20140710-sampitawala@yahoo.com-9NN3QM5XFZ.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20221509-D8KYBvpKeFL6bVtF7xS4.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20172909-X0cuOknHm1PtHX6J7tbf.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20191104-ranglin@gmail.com-JXEAIZJU6M.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20152801-tplatt@cantillon.com-5A7WTJS7ZI.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20172305-tuckerg@gmail.com-YNSKIKYLT0.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20192203-sal.mustafa@gmail.com-Z10IW1X1IU.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20141106-dBEA1u5uzsSqTseiVt0m.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20142304-fwC88MbJ5NEz

/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20151811-ZU1XdbCeJdrs5t7pYSyH.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20222306-all-RobAnglin@distro.hftconnect.com-U787G4LT0A.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20222306-all-GermanOspina@distro.hftconnect.com-2DKAXE8XP7.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20180203-swarmrb82@gmail.com-UZXBOT2FHB.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20231402-all-AndrewGoodman@distro.hftconnect.com-4ST77RCP5O.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20220712-all-BenjaminSaberin@distro.hftconnect.com-E0PUUVNM95.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20201704-all-EricMadeson@distro.hftconnect.com-K8BWDHCYSO.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20182012-EMquVchJF3cMLylr3ZEd.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20200704-all-DavidMcIntyre@distro.hftconne

/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20230903-4eWsIiuStq0XuQKBs7P5.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20220303-mqEMi1YZWg9t3Q4gqaOr.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20150603-1it0IO7zj8q04Ezdy4Bl.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20140105-xzpn87lF6nlN2Bg3OiM7.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20200704-MF9bNrS92AnXTYa1hvAA.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20202705-drucev@gmail.com-I87WN319ZV.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20210411-DS88TL7XBaqq1BGprWld.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20152907-cmoskowitz@deerfield.com-HZHGMSYS6E.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20142904-sal.mustafa@gmail.com-MP25L3HQUH.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20180903-OADb2XYj24LbYuU4SDVk.txt
/Users/drucev/notebooks/llamaindex/g

/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20201612-all-DanielleMalagrecaMartin@distro.hftconnect.com-QI8VCCLPXZ.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20181603-V4VZECvLqEOt7fNHgf0h.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20210302-0CtBlihYA4JiaspK9FSx.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20151208-IGesuRzgHO1jLkRLgCtR.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20152602-02HKaL75IGHZLTUptC6y.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20200503-all-EricMadeson@distro.hftconnect.com-MD0QLIQBI4.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20201703-all-NareshSingh@distro.hftconnect.com-WFPVY7SKD9.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20142703-tw4lX9lWcDUaCPqESaBv.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20221409-all-BenjaminSaberin@distro.hftconnect.com-C90BCB0FBX.txt
/Users/drucev/notebooks/llamaindex/

/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20190704-mwtXs7B48jI42iPnM8WQ.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20171304-dlund@hftconnect.com-XT2YDZU6TJ.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20203004-all-JeffMiller@distro.hftconnect.com-XCNI84PSBA.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20200903-JXAvDeBxldAh3IaOWaLX.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20201012-all-AngeloStornello@distro.hftconnect.com-G3JUIVS7I7.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20191104-bnGKmTTv2WyMtrIW8nUn.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20153108-nxWRoU6R9IrzaSxoQz7h.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20211305-all-RichardBottiglieri@distro.hftconnect.com-XJILPE0PNY.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20182407-iflores@stonepoint.com-40VDKAFG86.txt
/Users/drucev/notebooks/llamaindex/gpt_index/

...

/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20230303-Hf9SfvkXJrMWoQhmASPV.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20142103-alex.zhivov@gmail.com-QG5B9GV7HB.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20142908-lHd3hD1gmflpJ4CzE7U0.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20171511-johnbaio@hotmail.com-P02WOQRK2W.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20171101-UyxVICewFr4d4z9qK4Ab.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20212211-all-StephenKrug@distro.hftconnect.com-SRYBCNU7MR.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20150303-tucker@marinercapital.com-PMDJ089JAQ.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20182908-kJFIe1nZ78rgkKXU7LOZ.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20222109-all-AJFried@distro.hftconnect.com-EEACW1A2AP.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20142407-oj6D

/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20212704-all-DavidDinner@distro.hftconnect.com-2CJ3RR811N.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20152011-AcEsA8YcedTnpuEw3JwI.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20201304-HW8UsFZ2uHG4WRLJqZsL.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20211901-all-JohnDecker@distro.hftconnect.com-3LYILN38KB.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20222204-5K2G25PSg0iiPRL1mV52.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20172401-amhahn@gmail.com-7QH6DNBYJ8.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20150505-dlund@hftconnect.com-LSHJXK8YPY.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20221407-QzWKTO2YiPDjpljFJVQp.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20182201-95ZV8QORbPNnPGMj0VKe.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20152206-sidmsse@yahoo.com-VVXT

/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20211411-all-StevenKerstein@distro.hftconnect.com-QY8SN7YDCE.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20201603-AbJ4nWkcYqbvIDiMxoES.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20151706-bmoraller@kemnayusa.com-L2DBXYRXO9.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20180401-G0az9m5zJGjETU3LZnWD.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20172802-ddinner@obtix.net-25GVILDYQ1.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20202809-all-JonathanLevine@distro.hftconnect.com-UJKNMI8KQZ.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20200701-mXulSTDNLfWpWgT0KOWm.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20140205-sUrXi1CK8VRMGztanZMf.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20140901-prvs=40860ff0b9=bert.cohen@maverickcap.com-EVAF3O7Z6B.txt
/Users/drucev/notebooks/llamaindex/gpt_index/e

/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20180607-sal.mustafa@gmail.com-AGFOFO1EHD.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20222701-PG27OBoCPTKFsdpdFPFe.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20142602-gospina@gmail.com-AF1GDIWHD9.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20140502-nareshsingh@gmail.com-XHS7UCU1V2.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20141104-H31HRQcAuzlBhQTpRfDv.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20151509-zirxiv3QSbZU6g7TFmdw.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20151112-tony.cristiano@janapartners.com-ZWJ5R0MQYG.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20160202-Wz0ecZAanZny5ydQOHeh.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20190509-bmoraller@kemnayusa.com-BASUL6XY38.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20211304-drucev@gmail.com-1Y4UJCGBV8

/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20231901-ViYqaLFZhi0UjG4l5JdO.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20180903-ma.chuck@gmail.com-JGUZQQ2WET.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20171212-6vdaqAyTVzqr3Z9ToNVX.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20150210-kurt@brungardt.name-RI4N39LC9I.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20150210-sal.mustafa@gmail.com-QAA6FIFZAX.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20210402-X2QbbCLrNY2hno64aY3F.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20171010-tM93Ac8xSfa1SKtbjPSQ.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20171505-11lnUtXsMSEoxDwayLil.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20202606-all-EricSchnoor@distro.hftconnect.com-1ZBJZJE56O.txt
/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc/20210402-6Q2TymzYHFUjowCWU9Wy.txt
/Users/druc

In [4]:
embedding_model = 'text-embedding-ada-002'   # default

# split the documents into chunks
print(1, 'splitting', datetime.now())
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
# load embeddings into vector db
print(2, 'getting embeddings', datetime.now())
embeddings = OpenAIEmbeddings(model=embedding_model,
                              deployment = embedding_model,
)
# create the vectorestore to use as the index
db = Chroma.from_documents(texts, embeddings)
# expose this index in a retriever interface
print(4, 'instantiate retriever', datetime.now())
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})


1 splitting 2023-05-02 13:38:44.493895


Using embedded DuckDB without persistence: data will be transient


2 getting embeddings 2023-05-02 13:38:45.904285
4 instantiate retriever 2023-05-02 13:43:43.606870


In [5]:
qa_model = 'gpt-3.5-turbo-0301'

llm = ChatOpenAI(model_name=qa_model, 
                 temperature=0.2,
                )

qa = RetrievalQA.from_chain_type(llm=llm, 
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True
                                )


In [ ]:
# create a chain to answer questions
# print(5, datetime.now())
# qa = RetrievalQA.from_chain_type(llm=OpenAI(model_name=qa_model, 
#                                             temperature=0.5), 
#                                  chain_type="stuff", 
#                                  retriever=retriever, 
#                                  return_source_documents=True)


In [6]:
query = "what are some good endpoint protection vendors?"
print(6, datetime.now())
result = qa({"query": query})
result['result']

6 2023-05-02 13:43:56.811479


"The email doesn't provide a clear answer to that question, as it is asking for recommendations from the recipients. However, some popular endpoint protection vendors in the market are Symantec, McAfee, Trend Micro, Kaspersky, and Bitdefender. It's always a good idea to do your own research and evaluate different options based on your specific needs and budget."

In [7]:
query = "what are the names of some good Managed Service Providers?"
print(6, datetime.now())
result = qa({"query": query})
result['result']


6 2023-05-02 13:44:07.497881


'The email mentions three Managed Service Providers: AlphaServe, CDI, and Agio. However, it does not provide any opinions or reviews about them.'

In [12]:
query = "what are the names of leading Manged Security Service Providers or MDRs or MSSPs?"
print(6, datetime.now())
result = qa({"query": query})
result['result']


6 2023-05-02 13:46:43.470340


"There are several leading Managed Detection and Response (MDR) and Managed Security Service Providers (MSSPs) in the market. Some of the popular ones include:\n- CrowdStrike\n- FireEye\n- SecureWorks\n- Symantec\n- Trustwave\n- Rapid7\n- Carbon Black\n- Alert Logic\n- Arctic Wolf Networks\n- eSentire\n\nHowever, it's important to note that the best provider for your organization will depend on your specific needs and requirements. It's recommended to do thorough research and evaluations before selecting a provider."